In [5]:
import os
import wave
import json
import noisereduce as nr
import numpy as np
import tempfile
from vosk import Model, KaldiRecognizer
from difflib import get_close_matches

# Загрузка модели Vosk
model = Model("vosk-model-small-ru-0.22")
commands = [
    "отказ", "отмена", "подтверждение", "начать осаживание",
    "осадить на один вагон", "осадить на два вагона", "осадить на три вагона", "осадить на четыре вагона",
    "осадить на пять вагонов", "осадить на шесть вагонов", "осадить на семь вагонов",
    "осадить на восемь вагонов", "осадить на девять вагонов", "осадить на десять вагонов",
    "осадить на одиннадцать вагонов", "осадить на двенадцать вагонов", "осадить на тринадцать вагонов",
    "осадить на четырнадцать вагонов", "осадить на пятнадцать вагонов", "осадить на шестнадцать вагонов",
    "осадить на семнадцать вагонов", "осадить на восемнадцать вагонов", "осадить на девятнадцать вагонов",
    "осадить на двадцать вагонов", "продолжаем осаживание", "зарядка тормозной магистрали",
    "вышел из межвагонного пространства", "продолжаем роспуск", "растянуть автосцепки", "протянуть на один вагон",
    "протянуть на два вагона", "протянуть на три вагона", "протянуть на четыре вагона", "протянуть на пять вагонов",
    "протянуть на шесть вагонов", "протянуть на семь вагонов", "протянуть на восемь вагонов", "протянуть на девять вагонов",
    "протянуть на десять вагонов", "протянуть на одиннадцать вагонов", "протянуть на двенадцать вагонов",
    "протянуть на тринадцать вагонов", "протянуть на четырнадцать вагонов", "протянуть на пятнадцать вагонов",
    "протянуть на шестнадцать вагонов", "протянуть на семнадцать вагонов", "протянуть на восемнадцать вагонов",
    "протянуть на девятнадцать вагонов", "протянуть на двадцать вагонов", "отцепка", "назад на башмак",
    "захожу в межвагонное пространство", "остановка", "вперед на башмак", "сжать автосцепки", "назад с башмака",
    "тише", "вперед с башмака", "прекратить зарядку тормозной магистрали", "тормозить", "отпустить"
]

# Словарь для классификации команд и атрибутов
commands_dict = {
    1: ("отказ", -1),
    2: ("отмена", -1),
    3: ("подтверждение", -1),
    4: ("начать осаживание", -1),
    5: ("осадить", "вагон"),  # Осадить на количество вагонов, число будет определяться отдельно
    6: ("продолжаем осаживание", -1),
    7: ("зарядка тормозной магистрали", -1),
    8: ("вышел из межвагонного пространства", -1),
    9: ("продолжаем роспуск", -1),
    10: ("растянуть автосцепки", -1),
    11: ("протянуть", "вагон"),  # Протянуть на количество вагонов, число будет определяться отдельно
    12: ("отцепка", -1),
    13: ("назад на башмак", -1),
    14: ("захожу в межвагонное пространство", -1),
    15: ("остановка", -1),
    16: ("вперед на башмак", -1),
    17: ("сжать автосцепки", -1),
    18: ("назад с башмака", -1),
    19: ("тише", -1),
    20: ("вперед с башмака", -1),
    21: ("прекратить зарядку тормозной магистрали", -1),
    22: ("тормозить", -1),
    23: ("отпустить", -1)
}

# Функция для подавления шума с использованием noisereduce и временного файла
def remove_noise(input_file):
    # Открываем WAV файл
    with wave.open(input_file, 'rb') as wf:
        n_channels = wf.getnchannels()
        sample_rate = wf.getframerate()
        width = wf.getsampwidth()
        frames = wf.readframes(wf.getnframes())

    # Преобразование байтового потока в массив numpy
    audio_data = np.frombuffer(frames, dtype=np.int16)

    # Применяем подавление шума
    reduced_noise = nr.reduce_noise(y=audio_data, sr=sample_rate)

    # Создаем временный файл для хранения очищенного аудио
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")

    # Запись очищенного аудио во временный файл
    with wave.open(temp_file.name, 'wb') as wf_out:
        wf_out.setnchannels(n_channels)
        wf_out.setsampwidth(width)
        wf_out.setframerate(sample_rate)
        wf_out.writeframes(reduced_noise.astype(np.int16).tobytes())

    return temp_file.name  # Возвращаем путь к временно созданному файлу

# Функция для распознавания текста
def recognize_speech_from_file(file_path):
    wf = wave.open(file_path, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())
    result_text = ""

    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            result = rec.Result()
            result_dict = json.loads(result)
            if 'text' in result_dict and result_dict['text']:  
                result_text += result_dict['text'] + ' '

    return result_text.strip()

import re

# Словарь для преобразования текстовых чисел в цифры
text_to_number = {
    "один": 1, "два": 2, "три": 3, "четыре": 4, "пять": 5, "шесть": 6, "семь": 7,
    "восемь": 8, "девять": 9, "десять": 10, "одиннадцать": 11, "двенадцать": 12,
    "тринадцать": 13, "четырнадцать": 14, "пятнадцать": 15, "шестнадцать": 16,
    "семнадцать": 17, "восемнадцать": 18, "девятнадцать": 19, "двадцать": 20
}

# Функция для классификации команды и атрибута
def classify_command_and_attribute(recognized_text):
    # Проверяем, если команда содержит количество вагонов
    if "осадить" in recognized_text or "протянуть" in recognized_text:
        words = recognized_text.split()

        # Проверка на наличие числа в текстовом формате
        for word in words:
            if word.isdigit():  # Если в тексте есть числовое значение
                number = int(word)
                if "осадить" in recognized_text:
                    return 5, number  # Возвращаем номер команды "осадить" и число вагонов
                elif "протянуть" in recognized_text:
                    return 11, number  # Возвращаем номер команды "протянуть" и число вагонов
            elif word in text_to_number:  # Если число записано текстом
                number = text_to_number[word]
                if "осадить" in recognized_text:
                    return 5, number  # Возвращаем номер команды "осадить" и число вагонов
                elif "протянуть" in recognized_text:
                    return 11, number  # Возвращаем номер команды "протянуть" и число вагонов
    else:
        # Проверка остальных команд
        for key, value in commands_dict.items():
            if value[0] in recognized_text:
                return key, value[1]  # Возвращаем номер команды и атрибут -1

    return 0, -1  # Если команда не распознана, возвращаем 0 и -1

# Функция для обработки всех файлов в папке
def process_audio_files_in_directory(directory_path):
    for filename in os.listdir(directory_path):
        if filename.endswith(".wav"):
            file_path = os.path.join(directory_path, filename)

            # Шумоподавление
            denoised_file = remove_noise(file_path)

            try:
                # Распознавание речи
                text = recognize_speech_from_file(denoised_file)

                # Фильтрация результата
                recognized_commands = get_close_matches(text.strip(), commands, n=1, cutoff=0.6)

                if recognized_commands:
                    command_text = recognized_commands[0]
                    command, attribute = classify_command_and_attribute(command_text)
                    print(f"Файл: {filename}, Распознанный текст: {command_text}")
                    print(f"Команда: {command}, Атрибут: {attribute}")
                else:
                    print(f"Файл: {filename}, Распознанный текст: {text}")
                    print("Команда не распознана")

            finally:
                os.remove(denoised_file)

# Пример использования
directory_path = 'luga/03_07_2023'
process_audio_files_in_directory(directory_path)

Файл: 3f9867c1-846e-11ee-b078-c09bf4619c03.wav, Распознанный текст: назад на башмак
Команда: 13, Атрибут: -1
Файл: 3f988668-846e-11ee-9b41-c09bf4619c03.wav, Распознанный текст: подтверждение
Команда: 3, Атрибут: -1
Файл: 3f989900-846e-11ee-a3c2-c09bf4619c03.wav, Распознанный текст: растянуть автосцепки
Команда: 10, Атрибут: -1
Файл: 3f98aa99-846e-11ee-a159-c09bf4619c03.wav, Распознанный текст: назад на башмак
Команда: 13, Атрибут: -1
Файл: 3f98bbc6-846e-11ee-a454-c09bf4619c03.wav, Распознанный текст: отцепка
Команда: 12, Атрибут: -1
Файл: 3f98cb34-846e-11ee-b5c9-c09bf4619c03.wav, Распознанный текст: захожу в межвагонное пространство
Команда: 14, Атрибут: -1
Файл: 3f98da18-846e-11ee-9086-c09bf4619c03.wav, Распознанный текст: вышел из межвагонного пространства
Команда: 8, Атрибут: -1
Файл: 3f98e8bd-846e-11ee-b80c-c09bf4619c03.wav, Распознанный текст: подтверждение
Команда: 3, Атрибут: -1
Файл: 3f98f779-846e-11ee-b891-c09bf4619c03.wav, Распознанный текст: зарядка тормозной магистрали
Кома

PermissionError: [WinError 32] Процесс не может получить доступ к файлу, так как этот файл занят другим процессом: 'C:\\Users\\mikha\\AppData\\Local\\Temp\\tmpgsdotqex.wav'